<a href="https://colab.research.google.com/github/nannthd/project_AIEngineer/blob/main/AllModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow

#Resnet50

##original

###224

In [ ]:
import zipfile
import os

def unzip_file(zip_path, extract_to_folder):
    # ตรวจสอบว่ามีโฟลเดอร์ปลายทางหรือไม่ ถ้าไม่มีก็สร้างใหม่
    if not os.path.exists(extract_to_folder):
        os.makedirs(extract_to_folder)

    # เปิดไฟล์ ZIP และแตกไฟล์
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f'ไฟล์ถูกแตกที่ {extract_to_folder}')

# ตัวอย่างการใช้งาน
zip_path = '/content/drive/MyDrive/drug/FineTune/Resnet50/Vector_Resnet50_224.zip'  # ใส่เส้นทางไฟล์ ZIP ของคุณที่นี่
extract_to_folder = '/content/Vector_Resnet50_224'  # ใส่เส้นทางโฟลเดอร์ที่ต้องการแตกไฟล์ที่นี่
unzip_file(zip_path, extract_to_folder)

ไฟล์ถูกแตกที่ /content/Vector_Resnet50_224


In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image
import matplotlib.pyplot as plt

# ใช้โมเดล ResNet50 ของ TensorFlow ในการทำ embedding
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

def image_embedding(path):
    img = Image.open(path)
    img = img.resize((224, 224))  # ปรับขนาดภาพให้เข้ากับ ResNet50
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    curr_df = pd.DataFrame(preds[0]).T
    return curr_df

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes

def show_images(image_paths, title):
    plt.figure(figsize=(5, 5))
    plt.suptitle(title, fontsize=16)
    for i, path in enumerate(image_paths):
        img = Image.open(path)
        plt.subplot(1, len(image_paths), i + 1)
        plt.imshow(img)
        plt.axis('off')
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/Vector_Resnet50_224'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# โฟลเดอร์ที่เก็บภาพที่จะทดสอบ
test_image_folder = '/content/drive/MyDrive/drug/CorpTest'

# ประมวลผลภาพทั้งหมดในโฟลเดอร์
for image_file in os.listdir(test_image_folder):
    if image_file.endswith(('jpg', 'png')):
        image_path = os.path.join(test_image_folder, image_file)

        # ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
        similar_classes = find_most_similar_classes(image_path, all_embeddings)

        # แสดงผลลัพธ์
        print(f"Results for image: {image_file}")
        for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
            print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

        # แสดงภาพทดสอบ
        print("Showing test image:")
        show_images([image_path], "Test Image")

        # เพิ่มภาพที่คล้ายคลึงที่สุดในแต่ละคลาสที่พบ
        image_paths = []
        for class_name, _ in similar_classes:
            class_image_folder = os.path.join(csv_folder_path, class_name)
            if os.path.isdir(class_image_folder):
                image_files = [f for f in os.listdir(class_image_folder) if f.endswith(('jpg', 'png'))]
                if image_files:
                    top_image_path = os.path.join(class_image_folder, image_files[0])
                    image_paths.append(top_image_path)

        # แสดงภาพจากคลาสที่คล้ายคลึงที่สุด
        if image_paths:
            print("Showing similar images:")
            show_images(image_paths, "Similar Images")

        print("\n" + "="*40 + "\n")

ผิด1

ใหม่

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image
import matplotlib.pyplot as plt

# ใช้โมเดล ResNet50 ของ TensorFlow ในการทำ embedding
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

def image_embedding(path):
    img = Image.open(path)
    img = img.resize((224, 224))  # ปรับขนาดภาพให้เข้ากับ ResNet50
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    curr_df = pd.DataFrame(preds[0]).T
    return curr_df

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes, similarity_scores

def show_images(image_paths, title):
    plt.figure(figsize=(5, 5))
    plt.suptitle(title, fontsize=16)
    for i, path in enumerate(image_paths):
        img = Image.open(path)
        plt.subplot(1, len(image_paths), i + 1)
        plt.imshow(img)
        plt.axis('off')
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/Vector_Resnet50_224'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# โฟลเดอร์ที่เก็บภาพที่จะทดสอบ
test_image_folder = '/content/drive/MyDrive/drug/CorpTest'

# เก็บข้อมูล similarity ของภาพแต่ละภาพในตัวแปร
image_similarity_results = {}

# เก็บชื่อคลาสที่แท้จริงจากชื่อไฟล์
def get_class_from_filename(filename):
    return os.path.splitext(filename)[0]

true_classes = {}
for image_file in os.listdir(test_image_folder):
    if image_file.endswith(('jpg', 'png')):
        true_class = get_class_from_filename(image_file)
        true_classes[image_file] = true_class

# ประมวลผลภาพทั้งหมดในโฟลเดอร์
for image_file in os.listdir(test_image_folder):
    if image_file.endswith(('jpg', 'png')):
        image_path = os.path.join(test_image_folder, image_file)

        # ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
        similar_classes, similarity_scores = find_most_similar_classes(image_path, all_embeddings)

        # เก็บผลลัพธ์ในตัวแปร
        image_similarity_results[image_file] = {
            'similar_classes': similar_classes,
            'similarity_scores': similarity_scores
        }

        # แสดงผลลัพธ์
        print(f"Results for image: {image_file}")
        for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
            print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

        # แสดงภาพทดสอบ
        print("Showing test image:")
        show_images([image_path], "Test Image")

        print("\n" + "="*40 + "\n")

# สร้างกราฟแสดงผลการทำนาย
def plot_prediction_results(image_similarity_results, true_classes):
    for image_file, result in image_similarity_results.items():
        true_class = true_classes.get(image_file, None)
        if not true_class:
            continue

        similar_classes = result['similar_classes']
        is_correct = [1 if class_name == true_class else 0 for class_name, _ in similar_classes]
        labels = [class_name for class_name, _ in similar_classes]
        scores = [score for _, score in similar_classes]

        # สร้างกราฟแท่งแสดงการทำนาย
        plt.figure(figsize=(10, 6))
        bars = plt.bar(labels, scores, color=['green' if correct else 'red' for correct in is_correct])

        # เพิ่มข้อความแสดงว่าถูกหรือผิดบนกราฟ
        for bar, correct in zip(bars, is_correct):
            yval = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2, yval + 0.01, 'ถูก' if correct else 'ผิด', ha='center', fontsize=12)

        # กำหนดรายละเอียดกราฟ
        plt.title(f'การทำนายสำหรับ {image_file}')
        plt.ylabel('คะแนนความมั่นใจ')
        plt.ylim(0, 1)  # ตั้งค่าสูงสุดของแกน y เป็น 1
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()

        # แสดงกราฟ
        plt.show()

# แสดงกราฟผลการทำนาย
plot_prediction_results(image_similarity_results, true_classes)

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

def plot_heatmap(image_similarity_results, true_classes):
    all_counts = []
    image_files = []
    unique_classes = set()

    # เก็บชื่อคลาสทั้งหมดที่มี
    for image_file, result in image_similarity_results.items():
        similar_classes = result['similar_classes']
        labels = [class_name for class_name, _ in similar_classes]
        unique_classes.update(labels)

    unique_classes = sorted(unique_classes)  # เรียงชื่อคลาสตามตัวอักษร

    for image_file, result in image_similarity_results.items():
        true_class = true_classes.get(image_file, None)
        if not true_class:
            continue

        # หาคลาสที่มีคะแนนสูงสุด
        similar_classes = result['similar_classes']
        top_class = similar_classes[0][0]  # คลาสที่มีคะแนนสูงสุด
        counts_dict = {class_name: 0 for class_name in unique_classes}
        counts_dict[top_class] = 1  # นับจำนวนภาพที่ทำนายคลาสนี้
        counts_dict['image_file'] = image_file
        all_counts.append(counts_dict)
        image_files.append(image_file)

    # สร้าง DataFrame และเติมค่า NaN ด้วย 0
    counts_df = pd.DataFrame(all_counts).set_index('image_file').fillna(0)
    counts_df = counts_df[unique_classes]  # จัดเรียงคอลัมน์ตามชื่อคลาส

    plt.figure(figsize=(15, 10))
    sns.heatmap(counts_df, annot=True, fmt='.0f', cmap='YlGnBu', linewidths=.5, cbar_kws={'label': 'Number of Images'})
    plt.title('Heatmap of Top Predicted Classes (Count of Images)')
    plt.xlabel('Class Names')
    plt.ylabel('Image Files')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# แสดง Heatmap สำหรับจำนวนภาพที่ทำนายคลาสสูงสุด
plot_heatmap(image_similarity_results, true_classes)

###640

In [ ]:
import zipfile
import os

def unzip_file(zip_path, extract_to_folder):
    # ตรวจสอบว่ามีโฟลเดอร์ปลายทางหรือไม่ ถ้าไม่มีก็สร้างใหม่
    if not os.path.exists(extract_to_folder):
        os.makedirs(extract_to_folder)

    # เปิดไฟล์ ZIP และแตกไฟล์
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f'ไฟล์ถูกแตกที่ {extract_to_folder}')

# ตัวอย่างการใช้งาน
zip_path = '/content/drive/MyDrive/drug/FineTune/Resnet50/Vector_Resnet50_640.zip'  # ใส่เส้นทางไฟล์ ZIP ของคุณที่นี่
extract_to_folder = '/content/Vector_Resnet50_640'  # ใส่เส้นทางโฟลเดอร์ที่ต้องการแตกไฟล์ที่นี่
unzip_file(zip_path, extract_to_folder)

ไฟล์ถูกแตกที่ /content/Vector_Resnet50_640


In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image
import matplotlib.pyplot as plt

# ใช้โมเดล ResNet50 ของ TensorFlow ในการทำ embedding
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

def image_embedding(path):
    img = Image.open(path)
    img = img.resize((640, 640))  # ปรับขนาดภาพให้เข้ากับ ResNet50
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    curr_df = pd.DataFrame(preds[0]).T
    return curr_df

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes

def show_images(image_paths, title):
    plt.figure(figsize=(5, 5))
    plt.suptitle(title, fontsize=16)
    for i, path in enumerate(image_paths):
        img = Image.open(path)
        plt.subplot(1, len(image_paths), i + 1)
        plt.imshow(img)
        plt.axis('off')
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/Vector_Resnet50_640'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# โฟลเดอร์ที่เก็บภาพที่จะทดสอบ
test_image_folder = '/content/drive/MyDrive/drug/CorpTest'

# ประมวลผลภาพทั้งหมดในโฟลเดอร์
for image_file in os.listdir(test_image_folder):
    if image_file.endswith(('jpg', 'png')):
        image_path = os.path.join(test_image_folder, image_file)

        # ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
        similar_classes = find_most_similar_classes(image_path, all_embeddings)

        # แสดงผลลัพธ์
        print(f"Results for image: {image_file}")
        for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
            print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

        # แสดงภาพทดสอบ
        print("Showing test image:")
        show_images([image_path], "Test Image")

        # เพิ่มภาพที่คล้ายคลึงที่สุดในแต่ละคลาสที่พบ
        image_paths = []
        for class_name, _ in similar_classes:
            class_image_folder = os.path.join(csv_folder_path, class_name)
            if os.path.isdir(class_image_folder):
                image_files = [f for f in os.listdir(class_image_folder) if f.endswith(('jpg', 'png'))]
                if image_files:
                    top_image_path = os.path.join(class_image_folder, image_files[0])
                    image_paths.append(top_image_path)

        # แสดงภาพจากคลาสที่คล้ายคลึงที่สุด
        if image_paths:
            print("Showing similar images:")
            show_images(image_paths, "Similar Images")

        print("\n" + "="*40 + "\n")

ผิด3

ใหม่

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image
import matplotlib.pyplot as plt

# ใช้โมเดล ResNet50 ของ TensorFlow ในการทำ embedding
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

def image_embedding(path):
    img = Image.open(path)
    img = img.resize((640, 640))  # ปรับขนาดภาพให้เข้ากับ ResNet50
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    curr_df = pd.DataFrame(preds[0]).T
    return curr_df

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes, similarity_scores

def show_images(image_paths, title):
    plt.figure(figsize=(5, 5))
    plt.suptitle(title, fontsize=16)
    for i, path in enumerate(image_paths):
        img = Image.open(path)
        plt.subplot(1, len(image_paths), i + 1)
        plt.imshow(img)
        plt.axis('off')
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/Vector_Resnet50_640'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# โฟลเดอร์ที่เก็บภาพที่จะทดสอบ
test_image_folder = '/content/drive/MyDrive/drug/CorpTest'

# เก็บข้อมูล similarity ของภาพแต่ละภาพในตัวแปร
image_similarity_results = {}

# เก็บชื่อคลาสที่แท้จริงจากชื่อไฟล์
def get_class_from_filename(filename):
    return os.path.splitext(filename)[0]

true_classes = {}
for image_file in os.listdir(test_image_folder):
    if image_file.endswith(('jpg', 'png')):
        true_class = get_class_from_filename(image_file)
        true_classes[image_file] = true_class

# ประมวลผลภาพทั้งหมดในโฟลเดอร์
for image_file in os.listdir(test_image_folder):
    if image_file.endswith(('jpg', 'png')):
        image_path = os.path.join(test_image_folder, image_file)

        # ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
        similar_classes, similarity_scores = find_most_similar_classes(image_path, all_embeddings)

        # เก็บผลลัพธ์ในตัวแปร
        image_similarity_results[image_file] = {
            'similar_classes': similar_classes,
            'similarity_scores': similarity_scores
        }

        # แสดงผลลัพธ์
        print(f"Results for image: {image_file}")
        for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
            print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

        # แสดงภาพทดสอบ
        print("Showing test image:")
        show_images([image_path], "Test Image")

        print("\n" + "="*40 + "\n")

# สร้างกราฟแสดงผลการทำนาย
def plot_prediction_results(image_similarity_results, true_classes):
    for image_file, result in image_similarity_results.items():
        true_class = true_classes.get(image_file, None)
        if not true_class:
            continue

        similar_classes = result['similar_classes']
        is_correct = [1 if class_name == true_class else 0 for class_name, _ in similar_classes]
        labels = [class_name for class_name, _ in similar_classes]
        scores = [score for _, score in similar_classes]

        # สร้างกราฟแท่งแสดงการทำนาย
        plt.figure(figsize=(10, 6))
        bars = plt.bar(labels, scores, color=['green' if correct else 'red' for correct in is_correct])

        # เพิ่มข้อความแสดงว่าถูกหรือผิดบนกราฟ
        for bar, correct in zip(bars, is_correct):
            yval = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2, yval + 0.01, 'ถูก' if correct else 'ผิด', ha='center', fontsize=12)

        # กำหนดรายละเอียดกราฟ
        plt.title(f'การทำนายสำหรับ {image_file}')
        plt.ylabel('คะแนนความมั่นใจ')
        plt.ylim(0, 1)  # ตั้งค่าสูงสุดของแกน y เป็น 1
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()

        # แสดงกราฟ
        plt.show()

# แสดงกราฟผลการทำนาย
plot_prediction_results(image_similarity_results, true_classes)

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

def plot_heatmap(image_similarity_results, true_classes):
    all_counts = []
    image_files = []
    unique_classes = set()

    # เก็บชื่อคลาสทั้งหมดที่มี
    for image_file, result in image_similarity_results.items():
        similar_classes = result['similar_classes']
        labels = [class_name for class_name, _ in similar_classes]
        unique_classes.update(labels)

    unique_classes = sorted(unique_classes)  # เรียงชื่อคลาสตามตัวอักษร

    for image_file, result in image_similarity_results.items():
        true_class = true_classes.get(image_file, None)
        if not true_class:
            continue

        # หาคลาสที่มีคะแนนสูงสุด
        similar_classes = result['similar_classes']
        top_class = similar_classes[0][0]  # คลาสที่มีคะแนนสูงสุด
        counts_dict = {class_name: 0 for class_name in unique_classes}
        counts_dict[top_class] = 1  # นับจำนวนภาพที่ทำนายคลาสนี้
        counts_dict['image_file'] = image_file
        all_counts.append(counts_dict)
        image_files.append(image_file)

    # สร้าง DataFrame และเติมค่า NaN ด้วย 0
    counts_df = pd.DataFrame(all_counts).set_index('image_file').fillna(0)
    counts_df = counts_df[unique_classes]  # จัดเรียงคอลัมน์ตามชื่อคลาส

    plt.figure(figsize=(15, 10))
    sns.heatmap(counts_df, annot=True, fmt='.0f', cmap='YlGnBu', linewidths=.5, cbar_kws={'label': 'Number of Images'})
    plt.title('Heatmap of Top Predicted Classes (Count of Images)')
    plt.xlabel('Class Names')
    plt.ylabel('Image Files')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# แสดง Heatmap สำหรับจำนวนภาพที่ทำนายคลาสสูงสุด
plot_heatmap(image_similarity_results, true_classes)

##Fine Tune

###224

In [ ]:
import zipfile
import os

def unzip_file(zip_path, extract_to_folder):
    # ตรวจสอบว่ามีโฟลเดอร์ปลายทางหรือไม่ ถ้าไม่มีก็สร้างใหม่
    if not os.path.exists(extract_to_folder):
        os.makedirs(extract_to_folder)

    # เปิดไฟล์ ZIP และแตกไฟล์
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f'ไฟล์ถูกแตกที่ {extract_to_folder}')

# ตัวอย่างการใช้งาน
zip_path = '/content/drive/MyDrive/drug/FineTune/Resnet50/Vector_ResNet50_FT224.zip'  # ใส่เส้นทางไฟล์ ZIP ของคุณที่นี่
extract_to_folder = '/content/Vector_ResNet50_FT224'  # ใส่เส้นทางโฟลเดอร์ที่ต้องการแตกไฟล์ที่นี่
unzip_file(zip_path, extract_to_folder)

ไฟล์ถูกแตกที่ /content/Vector_ResNet50_FT224


In [ ]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define a feature extractor model
class FeatureExtractor(nn.Module):
    def __init__(self, base_model):
        super(FeatureExtractor, self).__init__()
        self.base_model = base_model
        self.base_model.fc = nn.Identity()  # Remove the final classification layer

    def forward(self, x):
        return self.base_model(x)

# Initialize ResNet-50 model and feature extractor
resnet50_model = models.resnet50(pretrained=False)
num_classes = 50  # Replace with the actual number of classes from your training
resnet50_model.fc = nn.Linear(resnet50_model.fc.in_features, num_classes)  # Replace the final layer with your trained layer
model_path = '/content/drive/MyDrive/drug/FineTune/Resnet50/model_resnet50_224.pth'
resnet50_model.load_state_dict(torch.load(model_path))
feature_extractor = FeatureExtractor(resnet50_model).to(device)
feature_extractor.eval()  # Set the model to evaluation mode

# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def image_embedding(path):
    img = Image.open(path).convert('RGB')  # Convert image to RGB
    img = transform(img).unsqueeze(0).to(device)  # Apply transformation and add batch dimension

    with torch.no_grad():
        # Extract features from the penultimate layer
        features = feature_extractor(img)
        avg_embedding = features.squeeze().cpu().numpy()  # Convert to numpy array and remove batch dimension

    curr_df = pd.DataFrame(avg_embedding).T
    return curr_df

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    if not os.path.exists(csv_folder):
        raise FileNotFoundError(f"CSV folder {csv_folder} not found.")

    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # Use CSV file name as class name
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            if 'ID' in df.columns:
                # Drop ID column and keep only class embeddings
                embeddings_no_id = df.drop(['ID'], axis=1)
                all_embeddings[class_name] = embeddings_no_id
            else:
                raise ValueError(f"ID column not found in {csv_file}.")
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    if not os.path.isdir(new_image_path):
        raise FileNotFoundError(f"Directory {new_image_path} not found.")

    image_files = [os.path.join(new_image_path, f) for f in os.listdir(new_image_path) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
    results = {}

    for image_path in image_files:
        new_embedding = image_embedding(image_path)
        similarity_scores = {}

        for class_name, embeddings_df in all_embeddings.items():
            # Compute similarity between new image embedding and class embeddings
            similarity_score = cosine_similarity(new_embedding, embeddings_df)
            max_similarity_score = similarity_score.max()  # Get highest similarity score in class
            similarity_scores[class_name] = max_similarity_score

        sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
        top_5_similar_classes = sorted_similarity[:5]

        results[image_path] = top_5_similar_classes

    return results

def display_image_with_results(image_path, similar_classes):
    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Top 5 Similar Classes for {os.path.basename(image_path)}")
    plt.show()

# Load embeddings from CSV folder
csv_folder_path = '/content/Vector_ResNet50_FT224'
try:
    all_embeddings = load_embeddings_from_csv(csv_folder_path)
except (FileNotFoundError, ValueError) as e:
    print(e)
    all_embeddings = {}

# Directory containing images to test
new_image_folder_path = '/content/drive/MyDrive/drug/CorpTest'

if all_embeddings:
    # Find top 5 most similar classes for each image in the folder
    try:
        similar_classes_results = find_most_similar_classes(new_image_folder_path, all_embeddings)
        # Print results and display images
        for image_path, similar_classes in similar_classes_results.items():
            print(f"\nImage: {image_path}")
            print("Top 5 most similar classes:")
            for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
                print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")
            display_image_with_results(image_path, similar_classes)
    except FileNotFoundError as e:
        print(e)
else:
    print("No embeddings loaded, cannot perform similarity search.")

ผิด2

ใหม่

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

# Define the FeatureExtractor class to extract embeddings from the model
class FeatureExtractor(nn.Module):
    def __init__(self, base_model):
        super(FeatureExtractor, self).__init__()
        # Use ResNet without the final fully connected layer for feature extraction
        self.base_model = nn.Sequential(*list(base_model.children())[:-1])  # Exclude the final classification layer

    def forward(self, x):
        with torch.no_grad():
            features = self.base_model(x)
            features = features.view(features.size(0), -1)  # Flatten the features to 1D
        return features

# Initialize ResNet-50 model and feature extractor
device = torch.device('cpu')  # Explicitly use CPU
resnet50_model = models.resnet50(pretrained=False)
num_classes = 50  # Replace with the actual number of classes from your training
resnet50_model.fc = nn.Linear(resnet50_model.fc.in_features, num_classes)  # Replace the final layer with your trained layer
model_path = '/content/drive/MyDrive/drug/FineTune/Resnet50/model_resnet50_224.pth'
resnet50_model.load_state_dict(torch.load(model_path, map_location=device))
feature_extractor = FeatureExtractor(resnet50_model).to(device)
feature_extractor.eval()  # Set the model to evaluation mode

# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def image_embedding(path):
    img = Image.open(path).convert('RGB')
    img = transform(img).unsqueeze(0).to(device)  # Apply transformations and add batch dimension
    with torch.no_grad():
        embedding = feature_extractor(img)
    curr_df = pd.DataFrame(embedding.cpu().numpy())
    return curr_df

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # Use CSV filename as class name
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            embeddings_no_id = df.drop(['ID'], axis=1)
            # Verify the dimensions
            print(f"Loaded embeddings for class {class_name} with shape: {embeddings_no_id.shape}")
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # Find the highest similarity score within the class
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes, similarity_scores

def show_images(image_paths, title):
    plt.figure(figsize=(5, 5))
    plt.suptitle(title, fontsize=16)
    for i, path in enumerate(image_paths):
        img = Image.open(path)
        plt.subplot(1, len(image_paths), i + 1)
        plt.imshow(img)
        plt.axis('off')
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# Load embeddings from CSV folder
csv_folder_path = '/content/Vector_ResNet50_FT224'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# Test image folder path
test_image_folder = '/content/drive/MyDrive/drug/CorpTest'

# Store similarity results
image_similarity_results = {}

# Get true classes from filenames
def get_class_from_filename(filename):
    return os.path.splitext(filename)[0]

true_classes = {}
for image_file in os.listdir(test_image_folder):
    if image_file.endswith(('jpg', 'png')):
        true_class = get_class_from_filename(image_file)
        true_classes[image_file] = true_class

# Process all images in the folder
for image_file in os.listdir(test_image_folder):
    if image_file.endswith(('jpg', 'png')):
        image_path = os.path.join(test_image_folder, image_file)

        # Find the most similar classes
        similar_classes, similarity_scores = find_most_similar_classes(image_path, all_embeddings)

        # Store results
        image_similarity_results[image_file] = {
            'similar_classes': similar_classes,
            'similarity_scores': similarity_scores
        }

        # Print results
        print(f"Results for image: {image_file}")
        for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
            print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

        # Show test image
        print("Showing test image:")
        show_images([image_path], "Test Image")

        print("\n" + "="*40 + "\n")

# Plot prediction results
def plot_prediction_results(image_similarity_results, true_classes):
    for image_file, result in image_similarity_results.items():
        true_class = true_classes.get(image_file, None)
        if not true_class:
            continue

        similar_classes = result['similar_classes']
        is_correct = [1 if class_name == true_class else 0 for class_name, _ in similar_classes]
        labels = [class_name for class_name, _ in similar_classes]
        scores = [score for _, score in similar_classes]

        # Create bar plot for predictions
        plt.figure(figsize=(10, 6))
        bars = plt.bar(labels, scores, color=['green' if correct else 'red' for correct in is_correct])

        # Add text labels for correctness
        for bar, correct in zip(bars, is_correct):
            yval = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2, yval + 0.01, 'ถูก' if correct else 'ผิด', ha='center', fontsize=12)

        # Set plot details
        plt.title(f'การทำนายสำหรับ {image_file}')
        plt.ylabel('คะแนนความมั่นใจ')
        plt.ylim(0, 1)  # Set y-axis limit
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()

        # Show plot
        plt.show()

# Display prediction plots
plot_prediction_results(image_similarity_results, true_classes)

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

def plot_heatmap(image_similarity_results, true_classes):
    all_counts = []
    image_files = []
    unique_classes = set()

    # เก็บชื่อคลาสทั้งหมดที่มี
    for image_file, result in image_similarity_results.items():
        similar_classes = result['similar_classes']
        labels = [class_name for class_name, _ in similar_classes]
        unique_classes.update(labels)

    unique_classes = sorted(unique_classes)  # เรียงชื่อคลาสตามตัวอักษร

    for image_file, result in image_similarity_results.items():
        true_class = true_classes.get(image_file, None)
        if not true_class:
            continue

        # หาคลาสที่มีคะแนนสูงสุด
        similar_classes = result['similar_classes']
        top_class = similar_classes[0][0]  # คลาสที่มีคะแนนสูงสุด
        counts_dict = {class_name: 0 for class_name in unique_classes}
        counts_dict[top_class] = 1  # นับจำนวนภาพที่ทำนายคลาสนี้
        counts_dict['image_file'] = image_file
        all_counts.append(counts_dict)
        image_files.append(image_file)

    # สร้าง DataFrame และเติมค่า NaN ด้วย 0
    counts_df = pd.DataFrame(all_counts).set_index('image_file').fillna(0)
    counts_df = counts_df[unique_classes]  # จัดเรียงคอลัมน์ตามชื่อคลาส

    plt.figure(figsize=(15, 10))
    sns.heatmap(counts_df, annot=True, fmt='.0f', cmap='YlGnBu', linewidths=.5, cbar_kws={'label': 'Number of Images'})
    plt.title('Heatmap of Top Predicted Classes (Count of Images)')
    plt.xlabel('Class Names')
    plt.ylabel('Image Files')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# แสดง Heatmap สำหรับจำนวนภาพที่ทำนายคลาสสูงสุด
plot_heatmap(image_similarity_results, true_classes)

###640

In [ ]:
import zipfile
import os

def unzip_file(zip_path, extract_to_folder):
    # ตรวจสอบว่ามีโฟลเดอร์ปลายทางหรือไม่ ถ้าไม่มีก็สร้างใหม่
    if not os.path.exists(extract_to_folder):
        os.makedirs(extract_to_folder)

    # เปิดไฟล์ ZIP และแตกไฟล์
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f'ไฟล์ถูกแตกที่ {extract_to_folder}')

# ตัวอย่างการใช้งาน
zip_path = '/content/drive/MyDrive/drug/FineTune/Resnet50/Vector_ResNet50_FT640.zip'  # ใส่เส้นทางไฟล์ ZIP ของคุณที่นี่
extract_to_folder = '/content/Vector_ResNet50_FT640'  # ใส่เส้นทางโฟลเดอร์ที่ต้องการแตกไฟล์ที่นี่
unzip_file(zip_path, extract_to_folder)

ไฟล์ถูกแตกที่ /content/Vector_ResNet50_FT640


In [ ]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define a feature extractor model
class FeatureExtractor(nn.Module):
    def __init__(self, base_model):
        super(FeatureExtractor, self).__init__()
        self.base_model = base_model
        self.base_model.fc = nn.Identity()  # Remove the final classification layer

    def forward(self, x):
        return self.base_model(x)

# Initialize ResNet-50 model and feature extractor
resnet50_model = models.resnet50(pretrained=False)
num_classes = 50  # Replace with the actual number of classes from your training
resnet50_model.fc = nn.Linear(resnet50_model.fc.in_features, num_classes)  # Replace the final layer with your trained layer
model_path = '/content/drive/MyDrive/drug/FineTune/model_resnet50_640.pth'
resnet50_model.load_state_dict(torch.load(model_path))
feature_extractor = FeatureExtractor(resnet50_model).to(device)
feature_extractor.eval()  # Set the model to evaluation mode

# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
])

def image_embedding(path):
    img = Image.open(path).convert('RGB')  # Convert image to RGB
    img = transform(img).unsqueeze(0).to(device)  # Apply transformation and add batch dimension

    with torch.no_grad():
        # Extract features from the penultimate layer
        features = feature_extractor(img)
        avg_embedding = features.squeeze().cpu().numpy()  # Convert to numpy array and remove batch dimension

    curr_df = pd.DataFrame(avg_embedding).T
    return curr_df

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    if not os.path.exists(csv_folder):
        raise FileNotFoundError(f"CSV folder {csv_folder} not found.")

    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # Use CSV file name as class name
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            if 'ID' in df.columns:
                # Drop ID column and keep only class embeddings
                embeddings_no_id = df.drop(['ID'], axis=1)
                all_embeddings[class_name] = embeddings_no_id
            else:
                raise ValueError(f"ID column not found in {csv_file}.")
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    if not os.path.isdir(new_image_path):
        raise FileNotFoundError(f"Directory {new_image_path} not found.")

    image_files = [os.path.join(new_image_path, f) for f in os.listdir(new_image_path) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
    results = {}

    for image_path in image_files:
        new_embedding = image_embedding(image_path)
        similarity_scores = {}

        for class_name, embeddings_df in all_embeddings.items():
            # Compute similarity between new image embedding and class embeddings
            similarity_score = cosine_similarity(new_embedding, embeddings_df)
            max_similarity_score = similarity_score.max()  # Get highest similarity score in class
            similarity_scores[class_name] = max_similarity_score

        sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
        top_5_similar_classes = sorted_similarity[:5]

        results[image_path] = top_5_similar_classes

    return results

def display_image_with_results(image_path, similar_classes):
    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Top 5 Similar Classes for {os.path.basename(image_path)}")
    plt.show()

# Load embeddings from CSV folder
csv_folder_path = '/content/Vector_ResNet50_FT640'
try:
    all_embeddings = load_embeddings_from_csv(csv_folder_path)
except (FileNotFoundError, ValueError) as e:
    print(e)
    all_embeddings = {}

# Directory containing images to test
new_image_folder_path = '/content/drive/MyDrive/drug/CorpTest'

if all_embeddings:
    # Find top 5 most similar classes for each image in the folder
    try:
        similar_classes_results = find_most_similar_classes(new_image_folder_path, all_embeddings)
        # Print results and display images
        for image_path, similar_classes in similar_classes_results.items():
            print(f"\nImage: {image_path}")
            print("Top 5 most similar classes:")
            for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
                print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")
            display_image_with_results(image_path, similar_classes)
    except FileNotFoundError as e:
        print(e)
else:
    print("No embeddings loaded, cannot perform similarity search.")

ผิด6

ใหม่

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

# Define the FeatureExtractor class to extract embeddings from the model
class FeatureExtractor(nn.Module):
    def __init__(self, base_model):
        super(FeatureExtractor, self).__init__()
        # Use ResNet without the final fully connected layer for feature extraction
        self.base_model = nn.Sequential(*list(base_model.children())[:-1])  # Exclude the final classification layer

    def forward(self, x):
        with torch.no_grad():
            features = self.base_model(x)
            features = features.view(features.size(0), -1)  # Flatten the features to 1D
        return features

# Initialize ResNet-50 model and feature extractor
device = torch.device('cpu')  # Explicitly use CPU
resnet50_model = models.resnet50(pretrained=False)
num_classes = 50  # Replace with the actual number of classes from your training
resnet50_model.fc = nn.Linear(resnet50_model.fc.in_features, num_classes)  # Replace the final layer with your trained layer
model_path = '/content/drive/MyDrive/drug/FineTune/Resnet50/model_resnet50_640.pth'
resnet50_model.load_state_dict(torch.load(model_path, map_location=device))
feature_extractor = FeatureExtractor(resnet50_model).to(device)
feature_extractor.eval()  # Set the model to evaluation mode

# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
])

def image_embedding(path):
    img = Image.open(path).convert('RGB')
    img = transform(img).unsqueeze(0).to(device)  # Apply transformations and add batch dimension
    with torch.no_grad():
        embedding = feature_extractor(img)
    curr_df = pd.DataFrame(embedding.cpu().numpy())
    return curr_df

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # Use CSV filename as class name
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            embeddings_no_id = df.drop(['ID'], axis=1)
            # Verify the dimensions
            print(f"Loaded embeddings for class {class_name} with shape: {embeddings_no_id.shape}")
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # Find the highest similarity score within the class
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes, similarity_scores

def show_images(image_paths, title):
    plt.figure(figsize=(5, 5))
    plt.suptitle(title, fontsize=16)
    for i, path in enumerate(image_paths):
        img = Image.open(path)
        plt.subplot(1, len(image_paths), i + 1)
        plt.imshow(img)
        plt.axis('off')
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# Load embeddings from CSV folder
csv_folder_path = '/content/Vector_ResNet50_FT640'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# Test image folder path
test_image_folder = '/content/drive/MyDrive/drug/CorpTest'

# Store similarity results
image_similarity_results = {}

# Get true classes from filenames
def get_class_from_filename(filename):
    return os.path.splitext(filename)[0]

true_classes = {}
for image_file in os.listdir(test_image_folder):
    if image_file.endswith(('jpg', 'png')):
        true_class = get_class_from_filename(image_file)
        true_classes[image_file] = true_class

# Process all images in the folder
for image_file in os.listdir(test_image_folder):
    if image_file.endswith(('jpg', 'png')):
        image_path = os.path.join(test_image_folder, image_file)

        # Find the most similar classes
        similar_classes, similarity_scores = find_most_similar_classes(image_path, all_embeddings)

        # Store results
        image_similarity_results[image_file] = {
            'similar_classes': similar_classes,
            'similarity_scores': similarity_scores
        }

        # Print results
        print(f"Results for image: {image_file}")
        for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
            print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

        # Show test image
        print("Showing test image:")
        show_images([image_path], "Test Image")

        print("\n" + "="*40 + "\n")

# Plot prediction results
def plot_prediction_results(image_similarity_results, true_classes):
    for image_file, result in image_similarity_results.items():
        true_class = true_classes.get(image_file, None)
        if not true_class:
            continue

        similar_classes = result['similar_classes']
        is_correct = [1 if class_name == true_class else 0 for class_name, _ in similar_classes]
        labels = [class_name for class_name, _ in similar_classes]
        scores = [score for _, score in similar_classes]

        # Create bar plot for predictions
        plt.figure(figsize=(10, 6))
        bars = plt.bar(labels, scores, color=['green' if correct else 'red' for correct in is_correct])

        # Add text labels for correctness
        for bar, correct in zip(bars, is_correct):
            yval = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2, yval + 0.01, 'ถูก' if correct else 'ผิด', ha='center', fontsize=12)

        # Set plot details
        plt.title(f'การทำนายสำหรับ {image_file}')
        plt.ylabel('คะแนนความมั่นใจ')
        plt.ylim(0, 1)  # Set y-axis limit
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()

        # Show plot
        plt.show()

# Display prediction plots
plot_prediction_results(image_similarity_results, true_classes)

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

def plot_heatmap(image_similarity_results, true_classes):
    all_counts = []
    image_files = []
    unique_classes = set()

    # เก็บชื่อคลาสทั้งหมดที่มี
    for image_file, result in image_similarity_results.items():
        similar_classes = result['similar_classes']
        labels = [class_name for class_name, _ in similar_classes]
        unique_classes.update(labels)

    unique_classes = sorted(unique_classes)  # เรียงชื่อคลาสตามตัวอักษร

    for image_file, result in image_similarity_results.items():
        true_class = true_classes.get(image_file, None)
        if not true_class:
            continue

        # หาคลาสที่มีคะแนนสูงสุด
        similar_classes = result['similar_classes']
        top_class = similar_classes[0][0]  # คลาสที่มีคะแนนสูงสุด
        counts_dict = {class_name: 0 for class_name in unique_classes}
        counts_dict[top_class] = 1  # นับจำนวนภาพที่ทำนายคลาสนี้
        counts_dict['image_file'] = image_file
        all_counts.append(counts_dict)
        image_files.append(image_file)

    # สร้าง DataFrame และเติมค่า NaN ด้วย 0
    counts_df = pd.DataFrame(all_counts).set_index('image_file').fillna(0)
    counts_df = counts_df[unique_classes]  # จัดเรียงคอลัมน์ตามชื่อคลาส

    plt.figure(figsize=(15, 10))
    sns.heatmap(counts_df, annot=True, fmt='.0f', cmap='YlGnBu', linewidths=.5, cbar_kws={'label': 'Number of Images'})
    plt.title('Heatmap of Top Predicted Classes (Count of Images)')
    plt.xlabel('Class Names')
    plt.ylabel('Image Files')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# แสดง Heatmap สำหรับจำนวนภาพที่ทำนายคลาสสูงสุด
plot_heatmap(image_similarity_results, true_classes)

#ViTModel

##original

###224

In [ ]:
import zipfile
import os

def unzip_file(zip_path, extract_to_folder):
    # ตรวจสอบว่ามีโฟลเดอร์ปลายทางหรือไม่ ถ้าไม่มีก็สร้างใหม่
    if not os.path.exists(extract_to_folder):
        os.makedirs(extract_to_folder)

    # เปิดไฟล์ ZIP และแตกไฟล์
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f'ไฟล์ถูกแตกที่ {extract_to_folder}')

# ตัวอย่างการใช้งาน
zip_path = '/content/drive/MyDrive/drug/FineTune/VITModel/Vector_VITModel_224.zip'  # ใส่เส้นทางไฟล์ ZIP ของคุณที่นี่
extract_to_folder = '/content/Vector_VITModel_224'  # ใส่เส้นทางโฟลเดอร์ที่ต้องการแตกไฟล์ที่นี่
unzip_file(zip_path, extract_to_folder)

ไฟล์ถูกแตกที่ /content/Vector_VITModel_224


In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import torch
import matplotlib.pyplot as plt

# ใช้โมเดล ViT ของ Hugging Face ในการทำ embedding
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

# ย้ายโมเดลไปยัง GPU ถ้ามี
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def image_embedding(path):
    try:
        img = Image.open(path).convert('RGB').resize((224, 224))  # ปรับขนาดภาพให้เข้ากับ ViT
        inputs = processor(images=img, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        avg_embedding = last_hidden_states.mean(dim=1).squeeze().cpu().detach().numpy()  # ค่าเฉลี่ยของ hidden states
        curr_df = pd.DataFrame(avg_embedding).T
        return curr_df
    except Exception as e:
        print(f"Error processing image {path}: {e}")
        return pd.DataFrame()

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            # ดรอปคอลัมน์ ID และเก็บเฉพาะเวกเตอร์ของคลาส
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        # คำนวณค่า similarity ระหว่าง embedding ของรูปภาพใหม่กับ embeddings ของคลาส
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes

def process_images_in_folder(image_folder_path, all_embeddings):
    results = {}
    for image_file in os.listdir(image_folder_path):
        image_path = os.path.join(image_folder_path, image_file)
        if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            print(f"Processing image: {image_file}")
            similar_classes = find_most_similar_classes(image_path, all_embeddings)
            results[image_file] = (image_path, similar_classes)
    return results

def display_images_with_results(results):
    for image_file, (image_path, similar_classes) in results.items():
        img = Image.open(image_path)
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 1, 1)
        plt.imshow(img)
        plt.title(f"Image: {image_file}")
        plt.axis('off')

        print(f"\nImage: {image_file}")
        print("Top 5 most similar classes:")
        for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
            print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

        plt.show()

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/Vector_VITModel_224'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# โฟลเดอร์ที่เก็บรูปภาพที่ต้องการทดสอบ
image_folder_path = '/content/drive/MyDrive/drug/CorpTest'

# ค้นหาคลาสที่มีความคล้ายกันที่สุดสำหรับแต่ละภาพในโฟลเดอร์
results = process_images_in_folder(image_folder_path, all_embeddings)

# แสดงภาพพร้อมกับผลลัพธ์
display_images_with_results(results)

ผิด3

ใหม่

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import torch
import matplotlib.pyplot as plt

# ใช้โมเดล ViT ของ Hugging Face ในการทำ embedding
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

# ย้ายโมเดลไปยัง GPU ถ้ามี
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def image_embedding(path):
    try:
        img = Image.open(path).convert('RGB').resize((224, 224))  # ปรับขนาดภาพให้เข้ากับ ViT
        inputs = processor(images=img, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        avg_embedding = last_hidden_states.mean(dim=1).squeeze().cpu().detach().numpy()  # ค่าเฉลี่ยของ hidden states
        curr_df = pd.DataFrame(avg_embedding).T
        return curr_df
    except Exception as e:
        print(f"Error processing image {path}: {e}")
        return pd.DataFrame()

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes, similarity_scores

def show_images(image_paths, title):
    plt.figure(figsize=(5, 5))
    plt.suptitle(title, fontsize=16)
    for i, path in enumerate(image_paths):
        img = Image.open(path)
        plt.subplot(1, len(image_paths), i + 1)
        plt.imshow(img)
        plt.axis('off')
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/Vector_VITModel_224'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# โฟลเดอร์ที่เก็บภาพที่จะทดสอบ
test_image_folder = '/content/drive/MyDrive/drug/CorpTest'

# เก็บข้อมูล similarity ของภาพแต่ละภาพในตัวแปร
image_similarity_results = {}

# เก็บชื่อคลาสที่แท้จริงจากชื่อไฟล์
def get_class_from_filename(filename):
    return os.path.splitext(filename)[0]

true_classes = {}
for image_file in os.listdir(test_image_folder):
    if image_file.endswith(('jpg', 'png')):
        true_class = get_class_from_filename(image_file)
        true_classes[image_file] = true_class

# ประมวลผลภาพทั้งหมดในโฟลเดอร์
for image_file in os.listdir(test_image_folder):
    if image_file.endswith(('jpg', 'png')):
        image_path = os.path.join(test_image_folder, image_file)

        # ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
        similar_classes, similarity_scores = find_most_similar_classes(image_path, all_embeddings)

        # เก็บผลลัพธ์ในตัวแปร
        image_similarity_results[image_file] = {
            'similar_classes': similar_classes,
            'similarity_scores': similarity_scores
        }

        # แสดงผลลัพธ์
        print(f"Results for image: {image_file}")
        for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
            print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

        # แสดงภาพทดสอบ
        print("Showing test image:")
        show_images([image_path], "Test Image")

        print("\n" + "="*40 + "\n")

# สร้างกราฟแสดงผลการทำนาย
def plot_prediction_results(image_similarity_results, true_classes):
    for image_file, result in image_similarity_results.items():
        true_class = true_classes.get(image_file, None)
        if not true_class:
            continue

        similar_classes = result['similar_classes']
        is_correct = [1 if class_name == true_class else 0 for class_name, _ in similar_classes]
        labels = [class_name for class_name, _ in similar_classes]
        scores = [score for _, score in similar_classes]

        # สร้างกราฟแท่งแสดงการทำนาย
        plt.figure(figsize=(10, 6))
        bars = plt.bar(labels, scores, color=['green' if correct else 'red' for correct in is_correct])

        # เพิ่มข้อความแสดงว่าถูกหรือผิดบนกราฟ
        for bar, correct in zip(bars, is_correct):
            yval = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2, yval + 0.01, 'ถูก' if correct else 'ผิด', ha='center', fontsize=12)

        # กำหนดรายละเอียดกราฟ
        plt.title(f'การทำนายสำหรับ {image_file}')
        plt.ylabel('คะแนนความมั่นใจ')
        plt.ylim(0, 1)  # ตั้งค่าสูงสุดของแกน y เป็น 1
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()

        # แสดงกราฟ
        plt.show()

# แสดงกราฟผลการทำนาย
plot_prediction_results(image_similarity_results, true_classes)

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

def plot_heatmap(image_similarity_results, true_classes):
    all_counts = []
    image_files = []
    unique_classes = set()

    # เก็บชื่อคลาสทั้งหมดที่มี
    for image_file, result in image_similarity_results.items():
        similar_classes = result['similar_classes']
        labels = [class_name for class_name, _ in similar_classes]
        unique_classes.update(labels)

    unique_classes = sorted(unique_classes)  # เรียงชื่อคลาสตามตัวอักษร

    for image_file, result in image_similarity_results.items():
        true_class = true_classes.get(image_file, None)
        if not true_class:
            continue

        # หาคลาสที่มีคะแนนสูงสุด
        similar_classes = result['similar_classes']
        top_class = similar_classes[0][0]  # คลาสที่มีคะแนนสูงสุด
        counts_dict = {class_name: 0 for class_name in unique_classes}
        counts_dict[top_class] = 1  # นับจำนวนภาพที่ทำนายคลาสนี้
        counts_dict['image_file'] = image_file
        all_counts.append(counts_dict)
        image_files.append(image_file)

    # สร้าง DataFrame และเติมค่า NaN ด้วย 0
    counts_df = pd.DataFrame(all_counts).set_index('image_file').fillna(0)
    counts_df = counts_df[unique_classes]  # จัดเรียงคอลัมน์ตามชื่อคลาส

    plt.figure(figsize=(15, 10))
    sns.heatmap(counts_df, annot=True, fmt='.0f', cmap='YlGnBu', linewidths=.5, cbar_kws={'label': 'Number of Images'})
    plt.title('Heatmap of Top Predicted Classes (Count of Images)')
    plt.xlabel('Class Names')
    plt.ylabel('Image Files')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# แสดง Heatmap สำหรับจำนวนภาพที่ทำนายคลาสสูงสุด
plot_heatmap(image_similarity_results, true_classes)

###640

In [ ]:
import zipfile
import os

def unzip_file(zip_path, extract_to_folder):
    # ตรวจสอบว่ามีโฟลเดอร์ปลายทางหรือไม่ ถ้าไม่มีก็สร้างใหม่
    if not os.path.exists(extract_to_folder):
        os.makedirs(extract_to_folder)

    # เปิดไฟล์ ZIP และแตกไฟล์
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f'ไฟล์ถูกแตกที่ {extract_to_folder}')

# ตัวอย่างการใช้งาน
zip_path = '/content/drive/MyDrive/drug/FineTune/VITModel/Vector_VITModel_640.zip'  # ใส่เส้นทางไฟล์ ZIP ของคุณที่นี่
extract_to_folder = '/content/Vector_VITModel_640'  # ใส่เส้นทางโฟลเดอร์ที่ต้องการแตกไฟล์ที่นี่
unzip_file(zip_path, extract_to_folder)

ไฟล์ถูกแตกที่ /content/Vector_VITModel_640


In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import torch
import matplotlib.pyplot as plt

# ใช้โมเดล ViT ของ Hugging Face ในการทำ embedding
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

# ย้ายโมเดลไปยัง GPU ถ้ามี
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def image_embedding(path):
    try:
        img = Image.open(path).convert('RGB').resize((640, 640))  # ปรับขนาดภาพให้เข้ากับ ViT
        inputs = processor(images=img, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        avg_embedding = last_hidden_states.mean(dim=1).squeeze().cpu().detach().numpy()  # ค่าเฉลี่ยของ hidden states
        curr_df = pd.DataFrame(avg_embedding).T
        return curr_df
    except Exception as e:
        print(f"Error processing image {path}: {e}")
        return pd.DataFrame()

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            # ดรอปคอลัมน์ ID และเก็บเฉพาะเวกเตอร์ของคลาส
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        # คำนวณค่า similarity ระหว่าง embedding ของรูปภาพใหม่กับ embeddings ของคลาส
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes

def process_images_in_folder(image_folder_path, all_embeddings):
    results = {}
    for image_file in os.listdir(image_folder_path):
        image_path = os.path.join(image_folder_path, image_file)
        if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            print(f"Processing image: {image_file}")
            similar_classes = find_most_similar_classes(image_path, all_embeddings)
            results[image_file] = (image_path, similar_classes)
    return results

def display_images_with_results(results):
    for image_file, (image_path, similar_classes) in results.items():
        img = Image.open(image_path)
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 1, 1)
        plt.imshow(img)
        plt.title(f"Image: {image_file}")
        plt.axis('off')

        print(f"\nImage: {image_file}")
        print("Top 5 most similar classes:")
        for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
            print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

        plt.show()

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/drug50cls_ViTModel_640'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# โฟลเดอร์ที่เก็บรูปภาพที่ต้องการทดสอบ
image_folder_path = '/content/drive/MyDrive/drug/CorpTest'

# ค้นหาคลาสที่มีความคล้ายกันที่สุดสำหรับแต่ละภาพในโฟลเดอร์
results = process_images_in_folder(image_folder_path, all_embeddings)

# แสดงภาพพร้อมกับผลลัพธ์
display_images_with_results(results)

ผิด4

ใหม่

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import torch
import matplotlib.pyplot as plt

# ใช้โมเดล ViT ของ Hugging Face ในการทำ embedding
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

# ย้ายโมเดลไปยัง GPU ถ้ามี
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def image_embedding(path):
    try:
        img = Image.open(path).convert('RGB').resize((640, 640))  # ปรับขนาดภาพให้เข้ากับ ViT
        inputs = processor(images=img, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        avg_embedding = last_hidden_states.mean(dim=1).squeeze().cpu().detach().numpy()  # ค่าเฉลี่ยของ hidden states
        curr_df = pd.DataFrame(avg_embedding).T
        return curr_df
    except Exception as e:
        print(f"Error processing image {path}: {e}")
        return pd.DataFrame()

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes, similarity_scores

def show_images(image_paths, title):
    plt.figure(figsize=(5, 5))
    plt.suptitle(title, fontsize=16)
    for i, path in enumerate(image_paths):
        img = Image.open(path)
        plt.subplot(1, len(image_paths), i + 1)
        plt.imshow(img)
        plt.axis('off')
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/Vector_VITModel_640'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# โฟลเดอร์ที่เก็บภาพที่จะทดสอบ
test_image_folder = '/content/drive/MyDrive/drug/CorpTest'

# เก็บข้อมูล similarity ของภาพแต่ละภาพในตัวแปร
image_similarity_results = {}

# เก็บชื่อคลาสที่แท้จริงจากชื่อไฟล์
def get_class_from_filename(filename):
    return os.path.splitext(filename)[0]

true_classes = {}
for image_file in os.listdir(test_image_folder):
    if image_file.endswith(('jpg', 'png')):
        true_class = get_class_from_filename(image_file)
        true_classes[image_file] = true_class

# ประมวลผลภาพทั้งหมดในโฟลเดอร์
for image_file in os.listdir(test_image_folder):
    if image_file.endswith(('jpg', 'png')):
        image_path = os.path.join(test_image_folder, image_file)

        # ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
        similar_classes, similarity_scores = find_most_similar_classes(image_path, all_embeddings)

        # เก็บผลลัพธ์ในตัวแปร
        image_similarity_results[image_file] = {
            'similar_classes': similar_classes,
            'similarity_scores': similarity_scores
        }

        # แสดงผลลัพธ์
        print(f"Results for image: {image_file}")
        for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
            print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

        # แสดงภาพทดสอบ
        print("Showing test image:")
        show_images([image_path], "Test Image")

        print("\n" + "="*40 + "\n")

# สร้างกราฟแสดงผลการทำนาย
def plot_prediction_results(image_similarity_results, true_classes):
    for image_file, result in image_similarity_results.items():
        true_class = true_classes.get(image_file, None)
        if not true_class:
            continue

        similar_classes = result['similar_classes']
        is_correct = [1 if class_name == true_class else 0 for class_name, _ in similar_classes]
        labels = [class_name for class_name, _ in similar_classes]
        scores = [score for _, score in similar_classes]

        # สร้างกราฟแท่งแสดงการทำนาย
        plt.figure(figsize=(10, 6))
        bars = plt.bar(labels, scores, color=['green' if correct else 'red' for correct in is_correct])

        # เพิ่มข้อความแสดงว่าถูกหรือผิดบนกราฟ
        for bar, correct in zip(bars, is_correct):
            yval = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2, yval + 0.01, 'ถูก' if correct else 'ผิด', ha='center', fontsize=12)

        # กำหนดรายละเอียดกราฟ
        plt.title(f'การทำนายสำหรับ {image_file}')
        plt.ylabel('คะแนนความมั่นใจ')
        plt.ylim(0, 1)  # ตั้งค่าสูงสุดของแกน y เป็น 1
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()

        # แสดงกราฟ
        plt.show()

# แสดงกราฟผลการทำนาย
plot_prediction_results(image_similarity_results, true_classes)

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

def plot_heatmap(image_similarity_results, true_classes):
    all_counts = []
    image_files = []
    unique_classes = set()

    # เก็บชื่อคลาสทั้งหมดที่มี
    for image_file, result in image_similarity_results.items():
        similar_classes = result['similar_classes']
        labels = [class_name for class_name, _ in similar_classes]
        unique_classes.update(labels)

    unique_classes = sorted(unique_classes)  # เรียงชื่อคลาสตามตัวอักษร

    for image_file, result in image_similarity_results.items():
        true_class = true_classes.get(image_file, None)
        if not true_class:
            continue

        # หาคลาสที่มีคะแนนสูงสุด
        similar_classes = result['similar_classes']
        top_class = similar_classes[0][0]  # คลาสที่มีคะแนนสูงสุด
        counts_dict = {class_name: 0 for class_name in unique_classes}
        counts_dict[top_class] = 1  # นับจำนวนภาพที่ทำนายคลาสนี้
        counts_dict['image_file'] = image_file
        all_counts.append(counts_dict)
        image_files.append(image_file)

    # สร้าง DataFrame และเติมค่า NaN ด้วย 0
    counts_df = pd.DataFrame(all_counts).set_index('image_file').fillna(0)
    counts_df = counts_df[unique_classes]  # จัดเรียงคอลัมน์ตามชื่อคลาส

    plt.figure(figsize=(15, 10))
    sns.heatmap(counts_df, annot=True, fmt='.0f', cmap='YlGnBu', linewidths=.5, cbar_kws={'label': 'Number of Images'})
    plt.title('Heatmap of Top Predicted Classes (Count of Images)')
    plt.xlabel('Class Names')
    plt.ylabel('Image Files')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# แสดง Heatmap สำหรับจำนวนภาพที่ทำนายคลาสสูงสุด
plot_heatmap(image_similarity_results, true_classes)

##Fine Tune

###224

In [ ]:
import zipfile
import os

def unzip_file(zip_path, extract_to_folder):
    # ตรวจสอบว่ามีโฟลเดอร์ปลายทางหรือไม่ ถ้าไม่มีก็สร้างใหม่
    if not os.path.exists(extract_to_folder):
        os.makedirs(extract_to_folder)

    # เปิดไฟล์ ZIP และแตกไฟล์
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f'ไฟล์ถูกแตกที่ {extract_to_folder}')

# ตัวอย่างการใช้งาน
zip_path = '/content/drive/MyDrive/drug/FineTune/VITModel/model_VITModel_224.zip'  # ใส่เส้นทางไฟล์ ZIP ของคุณที่นี่
extract_to_folder = '/content/model_VITModel_224'  # ใส่เส้นทางโฟลเดอร์ที่ต้องการแตกไฟล์ที่นี่
unzip_file(zip_path, extract_to_folder)

ไฟล์ถูกแตกที่ /content/model_VITModel_224


In [ ]:
import zipfile
import os

def unzip_file(zip_path, extract_to_folder):
    # ตรวจสอบว่ามีโฟลเดอร์ปลายทางหรือไม่ ถ้าไม่มีก็สร้างใหม่
    if not os.path.exists(extract_to_folder):
        os.makedirs(extract_to_folder)

    # เปิดไฟล์ ZIP และแตกไฟล์
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f'ไฟล์ถูกแตกที่ {extract_to_folder}')

# ตัวอย่างการใช้งาน
zip_path = '/content/drive/MyDrive/drug/FineTune/VITModel/Vector_VITModel_FT224.zip'  # ใส่เส้นทางไฟล์ ZIP ของคุณที่นี่
extract_to_folder = '/content/Vector_VITModel_FT224'  # ใส่เส้นทางโฟลเดอร์ที่ต้องการแตกไฟล์ที่นี่
unzip_file(zip_path, extract_to_folder)

ไฟล์ถูกแตกที่ /content/Vector_VITModel_FT224


In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import torch
import matplotlib.pyplot as plt

# ใช้โมเดล ViT ของ Hugging Face ในการทำ embedding
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('/content/ViT_FT_model_v1/ViT_FT_model_v1')

# ย้ายโมเดลไปยัง GPU ถ้ามี
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def image_embedding(path):
    try:
        img = Image.open(path).convert('RGB').resize((224, 224))  # ปรับขนาดภาพให้เข้ากับ ViT
        inputs = processor(images=img, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        avg_embedding = last_hidden_states.mean(dim=1).squeeze().cpu().detach().numpy()  # ค่าเฉลี่ยของ hidden states
        curr_df = pd.DataFrame(avg_embedding).T
        return curr_df
    except Exception as e:
        print(f"Error processing image {path}: {e}")
        return pd.DataFrame()

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            # ดรอปคอลัมน์ ID และเก็บเฉพาะเวกเตอร์ของคลาส
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes_in_folder(folder_path, all_embeddings):
    similarity_results = {}

    for image_file in os.listdir(folder_path):
        if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, image_file)
            new_embedding = image_embedding(image_path)
            similarity_scores = {}

            for class_name, embeddings_df in all_embeddings.items():
                similarity_score = cosine_similarity(new_embedding, embeddings_df)
                max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
                similarity_scores[class_name] = max_similarity_score

            sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
            top_5_similar_classes = sorted_similarity[:5]

            similarity_results[image_path] = top_5_similar_classes

    return similarity_results

def display_images_with_classes(similarity_results):
    for image_path, similar_classes in similarity_results.items():
        img = Image.open(image_path)
        plt.figure(figsize=(10, 5))
        plt.imshow(img)
        plt.title(f"Image: {os.path.basename(image_path)}")
        plt.axis('off')
        plt.show()

        print(f"Top 5 most similar classes for {os.path.basename(image_path)}:")
        for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
            print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")
        print()

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/drug50cls_ViT-FT-model-v1'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# โฟลเดอร์ที่มีภาพที่ต้องการทดสอบ
folder_path = '/content/drive/MyDrive/drug/CorpTest'

# ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับสำหรับภาพในโฟลเดอร์
similarity_results = find_most_similar_classes_in_folder(folder_path, all_embeddings)

# แสดงภาพและผลลัพธ์
display_images_with_classes(similarity_results)

ใหม่

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import torch
import matplotlib.pyplot as plt

# ใช้โมเดล ViT ของ Hugging Face ในการทำ embedding
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('/content/model_VITModel_224')

# ย้ายโมเดลไปยัง GPU ถ้ามี
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def image_embedding(path):
    try:
        img = Image.open(path).convert('RGB').resize((224, 224))  # ปรับขนาดภาพให้เข้ากับ ViT
        inputs = processor(images=img, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        avg_embedding = last_hidden_states.mean(dim=1).squeeze().cpu().detach().numpy()  # ค่าเฉลี่ยของ hidden states
        curr_df = pd.DataFrame(avg_embedding).T
        return curr_df
    except Exception as e:
        print(f"Error processing image {path}: {e}")
        return pd.DataFrame()

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes, similarity_scores

def show_images(image_paths, title):
    plt.figure(figsize=(5, 5))
    plt.suptitle(title, fontsize=16)
    for i, path in enumerate(image_paths):
        img = Image.open(path)
        plt.subplot(1, len(image_paths), i + 1)
        plt.imshow(img)
        plt.axis('off')
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/Vector_VITModel_FT224'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# โฟลเดอร์ที่เก็บภาพที่จะทดสอบ
test_image_folder = '/content/drive/MyDrive/drug/CorpTest'

# เก็บข้อมูล similarity ของภาพแต่ละภาพในตัวแปร
image_similarity_results = {}

# เก็บชื่อคลาสที่แท้จริงจากชื่อไฟล์
def get_class_from_filename(filename):
    return os.path.splitext(filename)[0]

true_classes = {}
for image_file in os.listdir(test_image_folder):
    if image_file.endswith(('jpg', 'png')):
        true_class = get_class_from_filename(image_file)
        true_classes[image_file] = true_class

# ประมวลผลภาพทั้งหมดในโฟลเดอร์
for image_file in os.listdir(test_image_folder):
    if image_file.endswith(('jpg', 'png')):
        image_path = os.path.join(test_image_folder, image_file)

        # ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
        similar_classes, similarity_scores = find_most_similar_classes(image_path, all_embeddings)

        # เก็บผลลัพธ์ในตัวแปร
        image_similarity_results[image_file] = {
            'similar_classes': similar_classes,
            'similarity_scores': similarity_scores
        }

        # แสดงผลลัพธ์
        print(f"Results for image: {image_file}")
        for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
            print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

        # แสดงภาพทดสอบ
        print("Showing test image:")
        show_images([image_path], "Test Image")

        print("\n" + "="*40 + "\n")

# สร้างกราฟแสดงผลการทำนาย
def plot_prediction_results(image_similarity_results, true_classes):
    for image_file, result in image_similarity_results.items():
        true_class = true_classes.get(image_file, None)
        if not true_class:
            continue

        similar_classes = result['similar_classes']
        is_correct = [1 if class_name == true_class else 0 for class_name, _ in similar_classes]
        labels = [class_name for class_name, _ in similar_classes]
        scores = [score for _, score in similar_classes]

        # สร้างกราฟแท่งแสดงการทำนาย
        plt.figure(figsize=(10, 6))
        bars = plt.bar(labels, scores, color=['green' if correct else 'red' for correct in is_correct])

        # เพิ่มข้อความแสดงว่าถูกหรือผิดบนกราฟ
        for bar, correct in zip(bars, is_correct):
            yval = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2, yval + 0.01, 'ถูก' if correct else 'ผิด', ha='center', fontsize=12)

        # กำหนดรายละเอียดกราฟ
        plt.title(f'การทำนายสำหรับ {image_file}')
        plt.ylabel('คะแนนความมั่นใจ')
        plt.ylim(0, 1)  # ตั้งค่าสูงสุดของแกน y เป็น 1
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()

        # แสดงกราฟ
        plt.show()

# แสดงกราฟผลการทำนาย
plot_prediction_results(image_similarity_results, true_classes)

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

def plot_heatmap(image_similarity_results, true_classes):
    all_counts = []
    image_files = []
    unique_classes = set()

    # เก็บชื่อคลาสทั้งหมดที่มี
    for image_file, result in image_similarity_results.items():
        similar_classes = result['similar_classes']
        labels = [class_name for class_name, _ in similar_classes]
        unique_classes.update(labels)

    unique_classes = sorted(unique_classes)  # เรียงชื่อคลาสตามตัวอักษร

    for image_file, result in image_similarity_results.items():
        true_class = true_classes.get(image_file, None)
        if not true_class:
            continue

        # หาคลาสที่มีคะแนนสูงสุด
        similar_classes = result['similar_classes']
        top_class = similar_classes[0][0]  # คลาสที่มีคะแนนสูงสุด
        counts_dict = {class_name: 0 for class_name in unique_classes}
        counts_dict[top_class] = 1  # นับจำนวนภาพที่ทำนายคลาสนี้
        counts_dict['image_file'] = image_file
        all_counts.append(counts_dict)
        image_files.append(image_file)

    # สร้าง DataFrame และเติมค่า NaN ด้วย 0
    counts_df = pd.DataFrame(all_counts).set_index('image_file').fillna(0)
    counts_df = counts_df[unique_classes]  # จัดเรียงคอลัมน์ตามชื่อคลาส

    plt.figure(figsize=(15, 10))
    sns.heatmap(counts_df, annot=True, fmt='.0f', cmap='YlGnBu', linewidths=.5, cbar_kws={'label': 'Number of Images'})
    plt.title('Heatmap of Top Predicted Classes (Count of Images)')
    plt.xlabel('Class Names')
    plt.ylabel('Image Files')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# แสดง Heatmap สำหรับจำนวนภาพที่ทำนายคลาสสูงสุด
plot_heatmap(image_similarity_results, true_classes)

#CLIPModel

##original

###224

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

# Load the CLIP model and processor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# Load the CSV file containing the vectors
csv_file_path = '/content/drive/MyDrive/drug/FineTune/CLIPModel/class_vectors.csv'
df = pd.read_csv(csv_file_path)

# Extract the text labels and embeddings from the CSV
text_labels = df['TextLabel'].tolist()
image_paths = df['ImagePath'].tolist()
image_embeddings = df[[col for col in df.columns if 'ImageEmbed_' in col]].values
text_embeddings = df[[col for col in df.columns if 'TextEmbed_' in col]].values

# Convert embeddings from lists to numpy arrays
image_embeddings = np.array(image_embeddings)
text_embeddings = np.array(text_embeddings)

# Function to compare a new image with stored vectors
def compare_image_with_stored_vectors(new_image_path):
    # Open the new image
    image = Image.open(new_image_path)

    # Prepare the inputs for the CLIP model
    inputs = processor(text=text_labels, images=image, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Forward pass through the model
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the image embedding for the new image
    new_image_embedding = outputs.image_embeds.squeeze().cpu().numpy().reshape(1, -1)  # Ensure 2D array

    # Get the text embeddings for the new image
    new_text_embeddings = outputs.text_embeds.squeeze().cpu().numpy()  # Ensure 2D array

    # Print shapes of the new embeddings
    print(f"New image embedding shape: {new_image_embedding.shape}")
    print(f"New text embeddings shape: {new_text_embeddings.shape}")

    # Ensure dimensions match for cosine similarity
    if new_image_embedding.shape[1] != image_embeddings.shape[1]:
        raise ValueError("Dimension mismatch between new image embedding and stored image embeddings.")
    if new_text_embeddings.shape[1] != text_embeddings.shape[1]:
        raise ValueError("Dimension mismatch between new text embedding and stored text embeddings.")

    # Compute cosine similarity between new image embedding and stored image embeddings
    image_similarities = cosine_similarity(new_image_embedding, image_embeddings)

    # Compute cosine similarity between new text embedding and stored text embeddings
    text_similarities = cosine_similarity(new_text_embeddings, text_embeddings)

    # Combine results from both similarities
    combined_similarities = np.maximum(image_similarities, text_similarities)

    # Get indices based on combined similarities
    most_similar_combined_indices = np.argsort(-combined_similarities, axis=1)[0]

    # Get unique classes
    unique_classes = set()
    top_results = []

    # Collect top results avoiding duplicate class names
    for index in most_similar_combined_indices:
        label = text_labels[index]
        if label not in unique_classes:
            unique_classes.add(label)
            top_results.append((label, combined_similarities[0][index]))  # Fix index usage

        if len(top_results) >= 6:  # Collect more than 5 results to ensure we have enough
            break

    # Print the results from 2nd to 6th most similar
    if len(top_results) >= 6:
        print("\nRankings from 1st to 5th most similar classes:")
        for i in range(1, 6):  # Indices 1 to 5 are ranks 2 to 6
            label, similarity = top_results[i]
            print(f"{label}: {similarity:.4f}")
    elif len(top_results) > 1:
        print("\nRankings from 1st to last available:")
        for i in range(1, len(top_results)):  # Indices 1 to end are ranks 2 to last available
            label, similarity = top_results[i]
            print(f"{label}: {similarity:.4f}")
    else:
        print("\nNot enough results to show rankings from 1st to 5th.")

    # Display the new image
    plt.figure(figsize=(6, 6))
    plt.imshow(image)
    plt.title(f"New Image: {new_image_path}")
    plt.axis('off')
    plt.show()



# Process all images in the base folder
base_folder = '/content/drive/MyDrive/drug/CorpTest'  # Base folder containing images

for image_file in os.listdir(base_folder):
    image_path = os.path.join(base_folder, image_file)
    if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
        print(f"\nProcessing image: {image_path}")
        compare_image_with_stored_vectors(image_path)

ผิด10

ใหม่

#accuracy

In [ ]:
# จำนวนการทายที่ถูกต้องและจำนวนการทายทั้งหมดสำหรับแต่ละชุด
correct_predictions = [49, 47, 49, 45, 47, 46, 49, 50, 40]  # ตัวอย่างค่า
total_predictions = [50, 50, 50, 50, 50, 50, 50, 50, 50]    # จำนวนการทายทั้งหมด (ค่าเท่ากันทุกชุด)

# คำนวณความแม่นยำสำหรับแต่ละชุด
accuracies = [(correct / total) * 100 for correct, total in zip(correct_predictions, total_predictions)]

# แสดงผลลัพธ์
for i, accuracy in enumerate(accuracies):
    print(f"Accuracy for set {i+1}: {accuracy:.2f}%")

Accuracy for set 1: 98.00%
Accuracy for set 2: 94.00%
Accuracy for set 3: 98.00%
Accuracy for set 4: 90.00%
Accuracy for set 5: 94.00%
Accuracy for set 6: 92.00%
Accuracy for set 7: 98.00%
Accuracy for set 8: 100.00%
Accuracy for set 9: 80.00%
